In [1]:
import sage.all
import itertools
from pprint import pprint
import sage.groups

In [70]:
thegroup = [3,3,4]  # interpreted as SL(i)\times SL(j)\times SL(k) acting on \mathbb{C}^i\otimes \mathbb{C}^j\otimes \mathbb{C}^k
ambientdim = 0             # the dimension of the space we are working in
for ii in thegroup:
    ambientdim+=ii-1
print(ambientdim,'is the ambient dimension.')

possiblevertices = []
for ii in thegroup:     # fill out the possiblevertices array
    temparr = []
    for jj in range(ii-1):
        anothertemparr = [0]*(ii-1) # ii-1 because there is n-1 free choices in the diagonal of SL(n)
        anothertemparr[jj] = 1
        temparr.append(tuple(anothertemparr))
    temparr.append(tuple([-1]*(ii-1)))

    possiblevertices.append(temparr)

# this is all the points we care about (in some nasty form to be dealt with later) - the product function just takes the set product over all of the elements.
thepoints = list(itertools.product(*possiblevertices))
#pprint(thepoints)
print(len(thepoints),"points.")

temp = []
for ii in range(len(thepoints)):
    blahblah = []
    for jj in range(len(thepoints[ii])):
        for kk in range(len(thepoints[ii][jj])):
            blahblah.append(thepoints[ii][jj][kk])
    temp.append(tuple(blahblah))
thepoints = temp

print(thepoints)

7 is the ambient dimension.
36 points.
[(1, 0, 1, 0, 1, 0, 0), (1, 0, 1, 0, 0, 1, 0), (1, 0, 1, 0, 0, 0, 1), (1, 0, 1, 0, -1, -1, -1), (1, 0, 0, 1, 1, 0, 0), (1, 0, 0, 1, 0, 1, 0), (1, 0, 0, 1, 0, 0, 1), (1, 0, 0, 1, -1, -1, -1), (1, 0, -1, -1, 1, 0, 0), (1, 0, -1, -1, 0, 1, 0), (1, 0, -1, -1, 0, 0, 1), (1, 0, -1, -1, -1, -1, -1), (0, 1, 1, 0, 1, 0, 0), (0, 1, 1, 0, 0, 1, 0), (0, 1, 1, 0, 0, 0, 1), (0, 1, 1, 0, -1, -1, -1), (0, 1, 0, 1, 1, 0, 0), (0, 1, 0, 1, 0, 1, 0), (0, 1, 0, 1, 0, 0, 1), (0, 1, 0, 1, -1, -1, -1), (0, 1, -1, -1, 1, 0, 0), (0, 1, -1, -1, 0, 1, 0), (0, 1, -1, -1, 0, 0, 1), (0, 1, -1, -1, -1, -1, -1), (-1, -1, 1, 0, 1, 0, 0), (-1, -1, 1, 0, 0, 1, 0), (-1, -1, 1, 0, 0, 0, 1), (-1, -1, 1, 0, -1, -1, -1), (-1, -1, 0, 1, 1, 0, 0), (-1, -1, 0, 1, 0, 1, 0), (-1, -1, 0, 1, 0, 0, 1), (-1, -1, 0, 1, -1, -1, -1), (-1, -1, -1, -1, 1, 0, 0), (-1, -1, -1, -1, 0, 1, 0), (-1, -1, -1, -1, 0, 0, 1), (-1, -1, -1, -1, -1, -1, -1)]


In [71]:
candidatetwoflags = list(itertools.product(range(thegroup[0]+1),repeat = thegroup[1]))
twoflags = []

def validtwoflag(aflag):
    for ii in range(thegroup[1]-1):
        if aflag[ii]<aflag[ii+1]: 
            return False
    return True
    
for ff in candidatetwoflags:
    if validtwoflag(ff):
        twoflags.append(ff)
print(len(twoflags))
print(twoflags[5:10])

def twoflagleq(aflag,bflag):
    for ii in range(thegroup[1]):
        if aflag[ii]>bflag[ii]:
            return False
    return True

def validthreeflag(aflag):
    for ii in range(thegroup[2]-1):
        if not twoflagleq(aflag[ii+1],aflag[ii]):
            return False
    return True

def threeflagleq(aflag,bflag):
    for ii in range(thegroup[2]):
        if not twoflagleq(aflag[ii],bflag[ii]):
            return False
    return True

candidatethreeflags = list(itertools.product(twoflags,repeat = thegroup[2]))
revthreeflags = []

for ff in candidatethreeflags:
    if validthreeflag(ff):
        revthreeflags.append(ff)
print(len(revthreeflags))

threeflags=list(revthreeflags[::-1])
print(threeflags[0])
print(threeflags[1000])
print(threeflags[4000])

def flagtocoeffs(aflag):
    outpoints = []
    for ii in range(thegroup[0]):
        for jj in range(thegroup[1]):
            for kk in range(thegroup[2]):
                if ii < aflag[kk][jj]:
                    outpoints.append((ii,jj,kk))
    return tuple(outpoints)

20
[(2, 1, 0), (2, 1, 1), (2, 2, 0), (2, 2, 1), (2, 2, 2)]
4116
((3, 3, 3), (3, 3, 3), (3, 3, 3), (3, 3, 3))
((3, 3, 2), (3, 3, 2), (3, 3, 1), (3, 3, 0))
((2, 1, 1), (2, 1, 1), (0, 0, 0), (0, 0, 0))


In [72]:
def pointconversion(tuple):
    return possiblevertices[0][tuple[0]]+possiblevertices[1][tuple[1]]+possiblevertices[2][tuple[2]]

def isunstable(inlocus):
    vertexset = []
    for ii in inlocus:
        vertexset.append(pointconversion(ii))
    hull = Polyhedron(vertexset)

    if hull.contains([0]*ambientdim): #returns False if 0 is in convex hull (i.e. if semistable with this basis)
        return False
    return True # returns True otherwise

unstableflags = []
def inabiggerone(inflag):
    for ff in unstableflags:
        if threeflagleq(inflag,ff): # returns True if "inflag" is contained in one of the already computed unstable flags
            return True
    return False # returns false otherwise


# just to be extra sure that the 'maximal' unstable flags that we foudn are actually maximal.
def maximalsanitycheck(inlocus):
    vertexset = []
    for ii in inlocus:
        vertexset.append(pointconversion(ii))
    hull = Polyhedron(vertexset)

    for pts in thepoints:
        if pts not in vertexset:
            huh = set(vertexset)
            huh.add(pts)
            thehull = Polyhedron(huh)
            if not thehull.contains([0]*ambientdim): # if it is not maximal :/
                print("If you are seeing this your code has bugs in it :(")
                print("Non-maximal unstable subspace:",vertexset)
                print("You can add the vector",pts)
                return False
    return True

for ff in threeflags:
    if not inabiggerone(ff): # if this isn't just a subcase of an already computed unstable locus, then we will find the convex hull
        if isunstable(flagtocoeffs(ff)):
            maximalsanitycheck(flagtocoeffs(ff))
            unstableflags.append(ff)

In [73]:
allmatrix = []
for ii in range(thegroup[0]):
    for jj in range(thegroup[2]):
        for kk in range(thegroup[1]):
            allmatrix.append((kk,ii,jj)) #0,1,2,

def printout(inflag):
    coeffs = flagtocoeffs(inflag)
    for ii in range(len(allmatrix)):
        if (ii)%thegroup[1] == 0:
            print("|" , end=" ")
        if allmatrix[ii] in coeffs:
            print("*",end=" ")
        else:
            print(" ", end=" ")
        if (ii+1)%(thegroup[1]*thegroup[2]) == 0:
            print('|')

def dot(u,v):
    outvalue = 0
    for ii in range(len(u)):
        outvalue += u[ii]*v[ii]
    return(outvalue)

def printweights(inweights):
    for ii in range(thegroup[0]):
        print('|', end=" ")
        for jj in range(thegroup[2]):
            for kk in range(thegroup[1]):
                theweight = inweights[0][ii]+inweights[1][kk]+inweights[2][jj]
                if theweight>0:
                    print('+',end=" ")
                if theweight ==0:
                    print(0,end=" ")
                if theweight<0:
                    print('-',end=" ")
            print('|', end=" ")
        print("")      


def findthe1ps(inflag):
    incoeffs = flagtocoeffs(inflag)
    vertexset = []
    for ii in incoeffs:
        vertexset.append(pointconversion(ii))
    hull = Polyhedron(vertexset)
    hyperplanes = hull.Hrepresentation()

    for hh in hyperplanes:
        hvec=hh.vector()
        ispos = dot(hvec[1:],list(vertexset)[0])>0
        problem=False
        shouldcontinue = True
        for pt in vertexset:
            if shouldcontinue:
                if dot(hvec[1:],pt)>0 and not ispos:
                    problem = True
                if dot(hvec[1:],pt)<0 and ispos:
                    problem = True
                if dot(hvec[1:],pt)==0:
                    problem = True
        if not problem:
            toprint=[(hvec[1],hvec[2],-hvec[1]-hvec[2]),(hvec[3],hvec[4],-hvec[3]-hvec[4]),(hvec[5],hvec[6],hvec[7],-hvec[5]-hvec[6]-hvec[7])]
            print(hh)
            print("Unstable by weights:",end="")
            print(toprint)
            #printweights(toprint) #uncomment this if you want the sanity check of computing the weights are actually positive
            printout(inflag)


print(len(unstableflags),"maximal unstable flags\n")
for ff in unstableflags:
    findthe1ps(ff)
    print()

18 maximal unstable flags

An equation (0, 0, 0, 0, 1, 1, 1) x - 1 == 0
Unstable by weights:[(0, 0, 0), (0, 0, 0), (1, 1, 1, -3)]
| * * * | * * * | * * * |       |
| * * * | * * * | * * * |       |
| * * * | * * * | * * * |       |

An inequality (8, -4, 8, -4, 9, 9, -3) x - 1 >= 0
Unstable by weights:[(8, -4, -4), (8, -4, -4), (9, 9, -3, -15)]
| * * * | * * * | * * * | *     |
| * * * | * * * | *     |       |
| * * * | * * * | *     |       |

An inequality (0, 0, 4, -2, 3, 3, -3) x - 1 >= 0
Unstable by weights:[(0, 0, 0), (4, -2, -2), (3, 3, -3, -3)]
| * * * | * * * | * * * | * * * |
| * * * | * * * |       |       |
| * * * | * * * |       |       |

An inequality (4, -2, 0, 0, 3, 3, -3) x - 1 >= 0
Unstable by weights:[(4, -2, -2), (0, 0, 0), (3, 3, -3, -3)]
| * * * | * * * | *     | *     |
| * * * | * * * | *     | *     |
| * * * | * * * | *     | *     |

An inequality (0, 0, 4, 4, 9, -3, -3) x - 1 >= 0
Unstable by weights:[(0, 0, 0), (4, 4, -8), (9, -3, -3, -3)]
| * * * | * * 

In [74]:
# How things are labelled in the thesis
if thegroup==[3,3,4]:
    print('(1)',end=" ")
    findthe1ps(unstableflags[0])
    print('(4)',end=" ")
    findthe1ps(unstableflags[1])
    print('(5)',end=" ")
    findthe1ps(unstableflags[2])
    print('(5 reflected)',end=" ")
    findthe1ps(unstableflags[3])
    print('(2)',end=" ")
    findthe1ps(unstableflags[4])
    print('(3)',end=" ")
    findthe1ps(unstableflags[5])
    print('(2 reflected)',end=" ")
    findthe1ps(unstableflags[6])
    print('(11 reflected)',end=" ")
    findthe1ps(unstableflags[7])
    print('(11)',end=" ")
    findthe1ps(unstableflags[8])
    print('(10)',end=" ")
    findthe1ps(unstableflags[9])
    print('(12)',end=" ")
    findthe1ps(unstableflags[10])
    print('(8)',end=" ")
    findthe1ps(unstableflags[11])
    print('(6)',end=" ")
    findthe1ps(unstableflags[12])
    print('(7)',end=" ")
    findthe1ps(unstableflags[13])
    print('(8 reflected)',end=" ")
    findthe1ps(unstableflags[14])
    print('(6 reflected)',end=" ")
    findthe1ps(unstableflags[15])
    print('(7 reflected)',end=" ")
    findthe1ps(unstableflags[16])
    print('(5)',end=" ")
    findthe1ps(unstableflags[17])

(1) An equation (0, 0, 0, 0, 1, 1, 1) x - 1 == 0
Unstable by weights:[(0, 0, 0), (0, 0, 0), (1, 1, 1, -3)]
| * * * | * * * | * * * |       |
| * * * | * * * | * * * |       |
| * * * | * * * | * * * |       |
(4) An inequality (8, -4, 8, -4, 9, 9, -3) x - 1 >= 0
Unstable by weights:[(8, -4, -4), (8, -4, -4), (9, 9, -3, -15)]
| * * * | * * * | * * * | *     |
| * * * | * * * | *     |       |
| * * * | * * * | *     |       |
(5) An inequality (0, 0, 4, -2, 3, 3, -3) x - 1 >= 0
Unstable by weights:[(0, 0, 0), (4, -2, -2), (3, 3, -3, -3)]
| * * * | * * * | * * * | * * * |
| * * * | * * * |       |       |
| * * * | * * * |       |       |
(5 reflected) An inequality (4, -2, 0, 0, 3, 3, -3) x - 1 >= 0
Unstable by weights:[(4, -2, -2), (0, 0, 0), (3, 3, -3, -3)]
| * * * | * * * | *     | *     |
| * * * | * * * | *     | *     |
| * * * | * * * | *     | *     |
(2) An inequality (0, 0, 4, 4, 9, -3, -3) x - 1 >= 0
Unstable by weights:[(0, 0, 0), (4, 4, -8), (9, -3, -3, -3)]
| * * * | * * *